In [1]:
from clean_data import run_cleaning_pipeline
from convert_parquet_to_json import convert_parquet_to_json
from mongo_import import import_json_to_mongodb
from logger import logger
#import os


In [9]:
INPUT_PATH = "../data/raw/fhvhv_tripdata_2021-10.parquet"
JSON_PATH = "../data/processed/trips.json"

columns_to_remove =[
    'originating_base_num', 
    'on_scene_datetime',
    'access_a_ride_flag'
]

columns_clean = [
    'hvfhs_license_num',
    'dispatching_base_num'
]

flag_cols = [
    'shared_request_flag',
    'shared_match_flag', 
    'wav_request_flag',	
    'wav_match_flag'
]

DB_NAME = "trips_db"
COLLECTION_NAME = "fhvhv_trips_2021-10"



In [ ]:
# print(os.getcwd())
# print(os.path.exists(INPUT_PATH))

In [3]:
# Step 1: Clean the data and get a DataFrame
df = run_cleaning_pipeline(INPUT_PATH, columns_to_remove, columns_clean, flag_cols)

c:\Users\dell\Downloads\projet BDD\fhv-tripdata-mongodb-optimization\src\clean_data.py:47: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[flag_cols]= df[flag_cols].replace({'N':0,'Y':1})


In [4]:
print(df)


         hvfhs_license_num dispatching_base_num    request_datetime  \
0                   HV0003               B02764 2021-10-01 00:28:52   
1                   HV0005               B03406 2021-10-01 00:55:47   
2                   HV0003               B02864 2021-10-01 00:28:02   
3                   HV0003               B02835 2021-10-01 00:15:17   
4                   HV0003               B02835 2021-10-01 00:30:53   
...                    ...                  ...                 ...   
16545351            HV0003               B03404 2021-10-31 23:45:48   
16545352            HV0003               B03404 2021-10-31 23:21:17   
16545353            HV0003               B03404 2021-10-31 23:49:35   
16545354            HV0005               B03406 2021-10-31 23:04:33   
16545355            HV0003               B03404 2021-10-31 23:29:26   

             pickup_datetime    dropoff_datetime  PULocationID  DOLocationID  \
0        2021-10-01 00:33:31 2021-10-01 01:07:51            68     

In [ ]:
df1 = df.iloc[0:6_000_000]

In [5]:
l = len(df)
l

16545356

In [ ]:
# Step 2: Convert the cleaned DataFrame to JSON Lines format

i = 0
batch_size = 1_000_000
count = 0
while i+batch_size < l:
    batch = df.iloc[i:i+batch_size]
    JSON_PATH = f"../data/processed/trips{count}.json"
    convert_parquet_to_json(batch, JSON_PATH)
    i += batch_size 
    count +=1


In [7]:
print(i)

16000000


In [18]:
SON_PATH = f"../data/processed/trips0.json"
import_json_to_mongodb(JSON_PATH, DB_NAME, COLLECTION_NAME)

connect_to_mongo() : An error occurred while connecting to MongoDB: Unknown option: uri. Did you mean one of (driver, journal, username) or maybe a camelCase version of one? Refer to docstring.


UnboundLocalError: cannot access local variable 'db' where it is not associated with a value

In [ ]:
# Step 3: Import the JSON Lines data into MongoDB
l = i / batch_size
for count in range(i):
    JSON_PATH = f"../data/processed/trips{count}.json"
    import_json_to_mongodb(JSON_PATH, DB_NAME, COLLECTION_NAME)